## Web Scrapping 
### 'Roadmap, Feedback Period: Artificial Intelligence - ethical and legal requirements' (*N* = 133) 
This code: 
- crawles data from the EC website (i.e., pdf-documents)
- extracts text data from the pdfs
- translates non-English text into English
- merges extracted text-from-pdf with meta-data

In [ ]:
pip install selenium

In [ ]:
pip install pdfplumber

In [ ]:
pip install deep-translator

In [ ]:
from selenium import webdriver             # importing selenium webdriver
from csv import writer                     # import writer for inserting data into csv later on
from bs4 import BeautifulSoup              # used to format or edit scraped data
from selenium.webdriver.chrome.options import Options
import time                                # wait upto 3 seconds when data is being scrapped

options = Options()
options.headless = True                    #get the data from EC site without opening goolge chrome window

In [ ]:
driver = webdriver.Chrome('C:/Users/batzdova/Desktop/chromedriver', options=options)       #location where the installed file for selenium web driver is.    
soup = list()
for i in range(0,14):                                                                     #with 31 p. to scrape containg 304 unique docs, will scrape each site for getting the links of all 304 pages. 
    if(i==0):
        driver.get("https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12527-Kunstliche-Intelligenz-ethische-und-rechtliche-Anforderungen/feedback_de?p_id=8242911")
    else:
        driver.get("https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12527-Kunstliche-Intelligenz-ethische-und-rechtliche-Anforderungen/feedback_de?p_id=8242911&page="+str(i))
    time.sleep(3)                                                                         #Used to add wait of 3 secs so that scrapping even on slow connection. Incase of >304 document than you must add wait of 4/ 5 secs.
    soup.append(BeautifulSoup(driver.page_source, 'html'))                                #Get the data into readable format and append it into soup list.
driver.quit() 

<ipython-input-25-26a654b260ec>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/batzdova/Desktop/chromedriver', options=options)       #location where the installed file for selenium web driver is.


In [ ]:
linkData = list()                              # variable for <a> tag reference.
# linkData = soup.find_all('a',{'class':'ecl-link'},href=True)
for i in soup:
    linkData.append(i.find_all('a',href=True)) # Get all the <a> tags from soup list.

In [15]:
# Get only the required references from links which start with F5
# for example a unique site has address https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12527-Artificial-intelligence-ethical-and-legal-requirements/F243432'
# And we need only the last string of this address for download and scraping document.
tagid = list()
for j in linkData:
    for i in j:
        test = i['href'].split('/')
        if test:
            if test[-1].startswith("F5"):
                tagid.append(test[-1])

In [ ]:
tagid

In [17]:
len(tagid)

133

### Data retrieval
##### + download each file from the EC site and rename it 
##### + save it into directory
###### (Run this code only when you haven't downloaded the files, yet.)

In [19]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import glob
from pathlib import Path

for idd in tagid: #call references for 304 page
    driver = webdriver.Chrome('C:/Users/batzdova/Desktop/chromedriver')
    driver.get('https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12527-Kunstliche-Intelligenz-ethische-und-rechtliche-Anforderungen/'+str(idd))
    try: # Download the specific file if found on a page with HTML CLASS NAME ecl-file__download
        download_file = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ecl-file__download"))) # check if class(downloading document) exist.
        driver.execute_script("arguments[0].click();", download_file) #click on class ecl-file__download
        time.sleep(3)
    except:
        continue
    PDF_DIR = Path("C:/Users/batzdova/Desktop/") #Path to save downloaded file.
    PDF_PATTERN = r'*.pdf'
    latest_file = max(PDF_DIR.glob(PDF_PATTERN), key=lambda f: f.stat().st_ctime,default=0)
    PDF_new_path = Path("C:/Users/batzdova/Desktop/" + str(idd + ".pdf"))
    os.rename(latest_file , PDF_new_path) # rename file and give a reference name to it for example if file name is 34knjo34n2nj34.pdf than that renamed it to F213422_en.pdf(reference name)

<ipython-input-19-315688472213>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/batzdova/Desktop/chromedriver')


###  Save Data into CSV

##### + save data it into a csv-file. 
##### + merge text from the downloaded pdf filewith meta-data in csv-file.

In [ ]:
import pdfplumber
from deep_translator import GoogleTranslator
data1 = list()
with open("scrap3.csv","w",newline="",encoding="utf-8") as f:                                                    # Ceate a new file if does not exist, or if exist rewrite data
    newWriter = writer(f) # Create new writeer
    header = ["Feedback reference","Submitted on","Submitted by","User type","Organisation","Organisation size",
              "Transparency register number","Country of origin","Initiative","Paragraph","pdf"]                 # Column Values
    newWriter.writerow(header) # Create Columns
    
    driver = webdriver.Chrome('C:/Users/batzdova/Desktop/chromedriver')                                          # driver location
    for idd in tagid: # call each unique reference for 304 pages
        driver.get('https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12527-Kunstliche-Intelligenz-ethische-und-rechtliche-Anforderungen/'+str(idd))
        time.sleep(3) 
        soupnew = BeautifulSoup(driver.page_source, 'html.parser')                                               # get all page content
        linkDatanew = soupnew.find_all("div", {"class": "ecl-u-mb-m"})                                           # get a div with class name ecl-u-mb-m
        linkpara = soupnew.find("p",{"class" : "ecl-u-type-paragraph-m"})                                        # get a div with class name ecl-u-type-paragraph-m for paragraph csv data
        linkcsv = str(soupnew.find_all("div", {"class":"ng-tns-c188-0"}))                                        # get a div with class name ng-tns-c188-0 for other columns e-g submitted-by
        # Finding the specific data in linkcsv variable, if not found than wrote - into that row
        infoList = list()
        for i in linkDatanew:
            infoList.append(str(i.get_text().strip()))
        infoList.append(linkpara.get_text() if linkpara else "-")
        if "Eingereicht am" not in linkcsv:
            infoList.insert(1,"-")
        if "Eingereicht von" not in linkcsv:
            infoList.insert(2,"-")
        if "Nutzertyp" not in linkcsv:
            infoList.insert(3,"-")
        if "Organisation" not in linkcsv:
            infoList.insert(4,"-")
        if "Organisationsgröße" not in linkcsv:
            infoList.insert(5,"-")
        if "Transparenzregisternummer" not in linkcsv:
            infoList.insert(6,"-")
        if "Herkunftsland" not in linkcsv:
            infoList.insert(7,"-")
        if "Initiative" not in linkcsv:
            infoList.insert(8,"-")
        
        try: # try method will extract text from pdf documents and save it into csv file
            with pdfplumber.open('C:/Users/batzdova/Desktop/Roadmap/files/'+str(idd)+'.pdf') as temp:
                data1 = list()
                translated = list()
                num = temp.pages
                for i,pg in enumerate(num):
                    page = temp.pages[i]
                    page1 = page.extract_text()
                    data1.append(page1)
                for i in data1:
                    translated.append(GoogleTranslator(source='auto', target='en').translate(i))
                print(temp)
                infoList.insert(10, translated) # save the document text into infoList
        except:
            continue
        finally:
            newWriter.writerow(infoList) # write the infoList into csv file.